In [1]:
import pandas as pd
import numpy as np
import torch
import random
from Model.RL_agent import DDPG
from Env.HIES_env_origin_tgcn import HIES_Env
from Utils.HIES_options import HIES_Options
from Utils import rl_utils
import matplotlib.pyplot as plt

In [ ]:
### 定义训练参数
hidden_dim   =  512
tau          =  0.003  # 软更新参数  0.001
buffer_size  =  100000
minimal_size =  10000
batch_size   =  1024   #64
sigma        =  0.02  # 高斯噪声标准差
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

### 创建训练环境
penalty_factor = 10
env_options = HIES_Options()
env = HIES_Env( env_options, penalty_factor )   ### 创建一个HEMS系统（训练环境）
seed = 42
rl_utils.set_seed(seed)

replay_buffer =  rl_utils.ReplayBuffer( buffer_size )
state_dim     = env.state_dim    ### 共有12维状态分量
action_dim    = env.action_dim   ### 共有8维行动分量

actor_lr = 0.0004
critic_lr = 0.005
gamma = 0.95
num_episodes = 10000
file_name = 'DDPG_model'
action_bound = 1
#scaled_action_indices = np.array([0, 1, 2, 3, 4, 5, 6, 7], dtype = np.int32)
scaled_action_indices = np.array([0, 1, 2, 4, 6], dtype = np.int32)


# TGCN
units = 5
stack_cnt = 2
time_step = env.K
horizon = 24
dropout_rate = 0.5
leaky_rate=0.2
#启动训练模式

agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, scaled_action_indices, sigma, actor_lr, critic_lr, tau, gamma, units, stack_cnt, time_step, horizon, dropout_rate, leaky_rate, device)  ### 创建训练agent
return_list, trace_list, violation_list, cost_list = rl_utils.train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size, program = 'train', seed=seed, file_name = file_name)

In [ ]:
episodes_list = list(range(len(return_list)))
plt.plot(episodes_list[500:], return_list[500:])
plt.xlabel('Episodes')
plt.ylabel('Returns')
#plt.title('DDPG on {}'.format(env_name))
plt.show()

my_return = rl_utils.moving_average(return_list, 99)
plt.plot(my_return[500:])
plt.xlabel('Episodes')
plt.ylabel('Returns')
#plt.title('DDPG on {}'.format(env_name))
plt.show()

In [ ]:
violation_list = np.array(violation_list)
plt.figure( figsize = (10, 4))
plt.plot(violation_list[1100:])
plt.xlabel('Episodes')
plt.ylabel('Violations')
#plt.title('DDPG on {}'.format(env_name))
plt.show()


violation_list = np.array(violation_list)
converged_violation = np.mean(violation_list[-100:])
print(converged_violation)

plt.figure( figsize = (10, 4))
my_violation = rl_utils.moving_average(violation_list, 99)
plt.plot(my_violation[1100:])
plt.xlabel('Episodes')
plt.ylabel('violations')
plt.title( f'converged_violation = {round(converged_violation*10)/10}')
#plt.title('DDPG on {}'.format(env_name))
plt.show()

In [ ]:
cost_list = np.array(cost_list)
converged_cost = np.mean(cost_list[-100:])
print(converged_cost)

plt.figure( figsize = (10, 4))
plt.plot(cost_list[1200:])
plt.xlabel('Episodes')
plt.ylabel('Cost')
#plt.title('DDPG on {}'.format(env_name))
plt.show()

plt.figure( figsize = (10, 4))
my_cost = rl_utils.moving_average(cost_list, 99)
plt.plot(my_cost[1200:])
plt.xlabel('Episodes')
plt.ylabel('Cost')
plt.title( f'converged_cost = {round(converged_cost*10)/10}')
#plt.title('DDPG on {}'.format(env_name))
plt.show()


In [44]:
###查看最后100个epoch的结果
import matplotlib.pyplot as plt
DAYS = 100
T    = 24
S_bss  = np.zeros( (DAYS, 24) )
S_hss  = np.zeros( (DAYS, 24) )
S_tes  = np.zeros( (DAYS, 24) )
S_css  = np.zeros( (DAYS, 24) )
P_bssc = np.zeros( (DAYS, 24) )
P_bssd = np.zeros( (DAYS, 24) )

m_B   =  np.zeros( (DAYS, 24) ) 
P_EL  =  np.zeros( (DAYS, 24) )
P_FC  =  np.zeros( (DAYS, 24) )
P_g   =  np.zeros( (DAYS, 24) )
g_AC  =  np.zeros( (DAYS, 24) )
g_FC  =  np.zeros( (DAYS, 24) )
g_EL  =  np.zeros( (DAYS, 24) )

g_tesc = np.zeros( (DAYS, 24) )
g_tesd = np.zeros( (DAYS, 24) )

q_cssc = np.zeros( (DAYS, 24) )
q_cssd = np.zeros( (DAYS, 24) )
m_B    = np.zeros( (DAYS, 24) )
P_EL   = np.zeros( (DAYS, 24) )
P_FC   = np.zeros( (DAYS, 24) )
P_CO   = np.zeros( (DAYS, 24) )
m_ch   = np.zeros( (DAYS, 24) )
m_dis  = np.zeros( (DAYS, 24) )
reward = np.zeros( (DAYS, 24) )
violation = np.zeros( (DAYS, 24), dtype = int)
days      = np.zeros(DAYS)

for day in range(DAYS):
    trace = trace_list[-day]
    days[day] = int(trace['day'][0])
    for t in range(24):
        S_bss[day][t]  =  trace['state'][t][3] 
        P_bssc[day][t] =  trace['decision'][t]['P_bssc']
        P_bssd[day][t] =  trace['decision'][t]['P_bssd']
        g_tesc[day][t] =  trace['decision'][t]['g_tesc']
        g_tesd[day][t] =  trace['decision'][t]['g_tesd']
        P_g[day][t]    =  trace['decision'][t]['P_g']
        P_CO[day][t]   =  trace['decision'][t]['P_CO']
        g_AC[day][t]   =  trace['decision'][t]['g_AC']
        g_FC[day][t]   =  trace['decision'][t]['g_FC']
        g_EL[day][t]   =  trace['decision'][t]['g_EL']

        q_cssc[day][t] =  trace['decision'][t]['q_cssc']
        q_cssd[day][t] =  trace['decision'][t]['q_cssd']
        m_B[day][t]    =  trace['decision'][t]['m_B']
        P_EL[day][t]   =  trace['decision'][t]['P_EL']
        P_FC[day][t]   =  trace['decision'][t]['P_FC']
        m_ch[day][t]   =  max( m_B[day][t] + P_EL[day][t] * env.env_options.k_EL - P_FC[day][t]/env.env_options.k_FC, 0 )
        m_dis[day][t]  = -min( m_B[day][t] + P_EL[day][t] * env.env_options.k_EL - P_FC[day][t]/env.env_options.k_FC, 0 ) 

        S_hss[day][t] =  trace['state'][t][4] 
        S_tes[day][t] =  trace['state'][t][5]
        S_css[day][t] =  trace['state'][t][6]
        reward[day][t] = trace['reward'][t]
        violation[day][t] = trace['violation'][t]

In [ ]:
plt.figure( figsize = (12, 6))

plt.subplots_adjust( wspace = 0.1, hspace = 0.1 )

plt.subplot(2, 2, 1)
plt.plot(S_bss.T, label = 'Batterty')
plt.title('battery (SOC)')
#plt.ylim([env_options.S_bss_min, env_options.S_bss_max])


plt.subplot(2, 2, 2)
plt.plot(S_hss.T, label = 'hydrogen tank')
plt.title('hydrogen tank (SOC)')
plt.ylim([env_options.S_hss_min, 50])

plt.subplot(2, 2, 3)
plt.plot(S_tes.T, label = 'hot water tank')
plt.title('hot water tank (SOC)')
#plt.ylim([env.env_options.S_tes_min, env_options.S_tes_max])


plt.subplot(2, 2, 4)
plt.plot(S_css.T, label = 'chilled water tank')
plt.title('chilled water tank (SOC)')
#plt.ylim([env_options.S_css_min, env_options.S_css_max])

plt.subplots_adjust( wspace = 0.3, hspace = 0.3 )

plt.show()

In [ ]:
trace_list[-50]['action'][-15]

In [ ]:
plt.figure( figsize = (12, 6))

plt.subplots_adjust( wspace = 0.1, hspace = 0.1 )

plt.subplot(2, 2, 1)
plt.plot(S_bss[13], label = 'Batterty')
plt.title('battery (SOC)')
#plt.ylim([env_options.S_bss_min, env_options.S_bss_max])


plt.subplot(2, 2, 2)
plt.plot(S_hss[13], label = 'hydrogen tank')
plt.title('hydrogen tank (SOC)')
plt.ylim([env_options.S_hss_min, 50])

plt.subplot(2, 2, 3)
plt.plot(S_tes[13], label = 'hot water tank')
plt.title('hot water tank (SOC)')
#plt.ylim([env.env_options.S_tes_min, env_options.S_tes_max])


plt.subplot(2, 2, 4)
plt.plot(S_css[13], label = 'chilled water tank')
plt.title('chilled water tank (SOC)')
#plt.ylim([env_options.S_css_min, env_options.S_css_max])

plt.subplots_adjust( wspace = 0.3, hspace = 0.3 )

plt.show()

In [ ]:
days = days.astype(int)
days

In [17]:
violation_ED = np.maximum( env_options.Q_ED[days] + P_EL + P_CO + P_bssc + P_bssd - P_g - env_options.P_solar_gen[days] - P_FC, 0)  ###电平衡约束违反情况
violation_HD = np.maximum(env_options.Q_HD[days] + g_tesc + g_tesd - g_FC - env_options.P_solar_heat[days] + g_AC, 0 )              ###热平衡约束违反情况
violation_CD = np.maximum( env_options.Q_CD[days] + q_cssc + q_cssd - g_AC * env_options.eta_AC, 0 )                                ###冷平衡约束违反情况

In [18]:
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
plt.figure( figsize = (15, 6) )

plt.subplot(2, 2, 1)

for i in range(5):
    plt.plot( violation[i], label = f'day{i}' )

plt.legend( loc = 'upper left' )
plt.title( 'violation' )
plt.ylim([0, 250])


plt.subplot(2, 2, 2)
for i in range(5):
    plt.plot( violation_ED[i], label = f'day{i}' )
plt.legend( loc = 'upper left' )
plt.title('violation_ED')
plt.ylim([0, 250])

plt.subplot(2, 2, 3)
for i in range(5):
    plt.plot( violation_HD[i], label = f'day{i}' )
plt.legend( loc = 'upper left' )
plt.title('violation_HD')
plt.ylim([0, 250])


plt.subplot(2, 2, 4)
for i in range(5):
    plt.plot( violation_CD[i], label = f'day{i}' )
plt.legend( loc = 'upper left' )
plt.title('violation_CD')
plt.ylim([0, 250])
plt.show()

In [ ]:
plt.figure( figsize = (15, 6) )

plt.subplot(2, 2, 1)
plt.plot( violation )
plt.title( 'violation' )
plt.ylim([0, 250])


plt.subplot(2, 2, 2)
plt.plot( violation_ED.T )
plt.title('violation_ED')
plt.ylim([0, 250])

plt.subplot(2, 2, 3)
plt.plot( violation_HD.T )
plt.title('violation_HD')
plt.ylim([0, 250])


plt.subplot(2, 2, 4)
plt.plot( violation_CD.T )
plt.title('violation_CD')
plt.ylim([0, 250])
plt.show()

In [ ]:
plt.figure( figsize = (15, 4))

plt.subplots_adjust(wspace = 0.1, hspace = 0.4)


plt.subplot(2, 2, 1)
plt.plot((P_bssc + P_bssd).T, label = 'charging and discharging'  )
#plt.ylim([-env_options.p_bssd_max, env_options.p_bssc_max])
plt.title('battery storage')

plt.subplot(2, 2, 2)
plt.plot( (m_B + P_EL * env.env_options.k_EL - P_FC/env.env_options.k_FC).T )
#plt.plot(m_ch, label = 'm_ch')
#plt.plot(m_dis, label = 'm_dis')
#plt.plot(m_B, label = 'm_B')
#plt.plot(P_EL*HEMS_Options.k_EL, label = 'P_EL')
#plt.plot(P_FC/HEMS_Options.k_FC, label = 'P_FC')
#plt.ylim([-env_options.m_hssd_max, env_options.m_hssc_max])
plt.title('hydrogen tank')
plt.legend( loc = 'upper center')



plt.subplot(2, 2, 3)
plt.plot( (g_tesc + g_tesd).T, label = 'charging and discharging'  )
#plt.ylim([-env_options.g_tesd_max, env_options.g_tesc_max])
plt.title('hot water tank')


plt.subplot(2, 2, 4)
plt.plot( (q_cssc + q_cssd).T, label = 'charging and discharging'  )
#plt.ylim([-env_options.q_cssd_max, env_options.q_cssd_max])
plt.title('chilled water tank')
plt.show()

In [ ]:
#plt.plot(env_options.P_solar_gen.flatten(), label = 'solar')
#plt.plot(env_options.Q_ED.flatten(), label = 'Q_ED')
#plt.plot(env_options.Q_HD.flatten(), label = 'Q_HD')
#plt.plot(env_options.Q_CD.faltten(), label = 'Q_CD')
#plt.legend(loc = 'upper left')
#plt.show()


plt.figure( figsize = (20, 6) )
plt.subplots_adjust( hspace = 0.6 )


plt.subplot(4, 1, 1)
plt.plot( env_options.P_solar_gen.flatten(), color = default_colors[0] )
plt.title( 'Solar generation' )


plt.subplot(4, 1, 2)
plt.plot( env_options.Q_ED.flatten(), color = default_colors[1] )
plt.title('Q_ED')


plt.subplot(4, 1, 3)
plt.plot( env_options.Q_HD.flatten(), color = default_colors[2] )
plt.title('Q_HD')

plt.subplot(4, 1, 4)
plt.plot( env_options.Q_CD.flatten(), color = default_colors[3] )
plt.title('Q_CD')



In [ ]:
plt.figure( figsize = (18, 10))
plt.subplot(3, 1, 1)
plt.plot(env_options.Q_ED[0:20].flatten(), color = default_colors[0])
plt.subplot(3, 1, 2)
plt.plot(env_options.Q_HD[0:20].flatten(), color = default_colors[1])
plt.subplot(3, 1, 3)
plt.plot(env_options.Q_CD[0:20].flatten(), color = default_colors[2])
plt.show()

In [ ]:
plt.figure(figsize = (12, 4))
plt.plot(P_g.T)
plt.title('P_g')
plt.show()

In [ ]:
plt.figure(figsize = (12, 4))
plt.plot(P_EL.T)
plt.title('P_EL')
plt.show()

In [ ]:
plt.figure(figsize = (12, 4))
plt.plot(P_FC.T)
plt.title('P_FC')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.subplot(3, 3, 1)
plt.plot(P_FC[i])
plt.title('P_FC')

plt.subplot(3, 3, 2)
plt.plot(env_options.P_solar_gen[days[i]])
plt.title('P_solar_gen')

plt.subplot(3, 3, 3)
plt.plot(env.env_options.Q_ED[days[i]])
plt.title('Q_ED')


plt.subplot(3, 3, 4)
plt.plot(P_CO[i])
plt.title('P_CO')

plt.subplot(3, 3, 5)
plt.plot(P_g[i])
plt.title('P_g')

plt.subplot(3, 3, 6)
plt.plot(P_EL[i])
plt.title('P_EL')

plt.subplot(3, 3, 7)
plt.plot(P_bssc[i])
plt.title('P_bssc')

plt.subplot(3, 3, 8)
plt.plot(P_bssd[i])
plt.title('P_bssd')

plt.subplot(3, 3, 9)
plt.plot(env_options.lambda_b[days[i]])
plt.title('Price')

plt.show()

In [ ]:
l = np.zeros(len(env_options.P_solar_heat[days[i]] + g_FC[i] - env.env_options.Q_HD[days[i]] - g_AC[i]))
plt.plot(env_options.P_solar_heat[days[i]] + g_FC[i] - env.env_options.Q_HD[days[i]] - g_AC[i])
plt.plot(l)
plt.title('Heating balance')
plt.show()


In [ ]:
plt.figure(figsize = (10, 8))
plt.subplot(2, 3, 1)
plt.plot(g_FC[i])
plt.title('g_FC')


plt.subplot(2, 3, 2)
plt.plot(env_options.P_solar_heat[days[i]])
plt.title('P_solar_heat')

plt.subplot(2, 3, 3)
plt.plot(env.env_options.Q_HD[days[i]])
plt.title('Q_HD')


plt.subplot(2, 3, 4)
plt.plot(g_AC[i])
plt.title('g_AC')

plt.subplot(2, 3, 5)
plt.plot(g_tesc[i])
plt.title('g_tesc')


plt.subplot(2, 3, 6)
plt.plot(g_tesd[i])
plt.title('g_tesd')

plt.show()

In [ ]:
l = np.zeros(len(env_options.P_solar_heat[days[i]] + g_FC[i] - env.env_options.Q_HD[days[i]] - g_AC[i]))
plt.plot(env_options.P_solar_heat[days[i]] + g_FC[i] - env.env_options.Q_HD[days[i]] - g_AC[i])
plt.plot(l)
plt.title('Heating balance')
plt.show()


In [ ]:
plt.figure(figsize = (10, 8))
plt.subplot(2, 3, 1)
plt.plot(q_cssc[i])
plt.title('q_cssc')


plt.subplot(2, 3, 2)
plt.plot(env_options.Q_CD[days[i]])
plt.title('Q_CD')

plt.subplot(2, 3, 3)
plt.plot(q_cssd[i])
plt.title('q_cssd')


plt.subplot(2, 3, 4)
plt.plot(g_AC[i] * env_options.eta_AC)
plt.title('q_AC')


plt.show()

In [50]:
hours = list(range(24))
bar_width = 0.5
P_g_buy = np.maximum(P_g, 0)
P_g_sell = np.minimum(P_g, 0)

P_solar_gen = env_options.P_solar_gen[-1]
Q_ED       = env_options.Q_ED[-1]
Q_CD      = env_options.Q_CD[-1]
Q_HD      = env_options.Q_HD[-1]
lambda_b = env_options.lambda_b[-1]
P_solar_heat = env_options.P_solar_heat[-1]
q_AC = g_AC * env_options.eta_AC


In [ ]:
# 电能来源与消耗策略
fig1, ax1 = plt.subplots(figsize=(10, 6), dpi=80)

bar1 = ax1.bar(hours, P_FC[-1], bar_width, label='$P^{FC}_t$(>0)', color='green')
bar2 = ax1.bar(hours, P_bssd[-1], bar_width, label='$P^{bssd}_t$(>0)', color='salmon', bottom=P_FC[-1])
bar3 = ax1.bar(hours, P_g_buy[-1], bar_width, label='$P^{g}_t$', color='orange', bottom=P_FC[-1] + P_bssd[-1])
bar4 = ax1.bar(hours, P_solar_gen, bar_width, label='PV Generation', color='skyblue', bottom=P_FC[-1] + P_bssd[-1]+P_g_buy[-1])

bar5 = ax1.bar(hours, -P_CO[-1], bar_width, label='$P^{CO}_t$(<0)', color='purple', bottom=0)
bar6 = ax1.bar(hours, -P_EL[-1], bar_width, label='$P^{EL}_t$(<0)', color='green', bottom=-P_CO[-1])
bar7 = ax1.bar(hours, -P_bssc[-1], bar_width, label='$P^{bssc}_t$(<0)', color='salmon', bottom=-P_EL[-1] - P_CO[-1])
bar8 = ax1.bar(hours, -P_g_sell[-1], bar_width, color='orange', bottom=-P_EL[-1] - P_CO[-1]-P_bssc[-1])


ax1.plot(hours, Q_ED, label='$Q^{ED}_t$', color='black')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Power (kW)')
ax1.set_title('Hourly electricity generation and consumption')
ax1.set_xticks(hours)
ax1.set_xticklabels(hours)
ax1.legend(loc='upper left')

ax12 = ax1.twinx()
ax12.plot(hours, lambda_b, label='Price', color='red', marker='o')
ax12.set_ylabel('Price/$')
ax12.legend(loc='upper right')

plt.show()

In [ ]:
# 热能来源与消耗策略
fig2, ax2 = plt.subplots(figsize=(10, 6), dpi=80)

bar2 = ax2.bar(hours, g_tesd[-1], bar_width, label='$g^{tesd}_t$(>0)', color='salmon', bottom=0)
bar3 = ax2.bar(hours, g_FC[-1], bar_width, label='$g^{FC}_t$(>0)', color='green', bottom=g_tesd[-1])
bar6 = ax2.bar(hours, g_EL[-1], bar_width, label='$g^{EL}_t$(>0)', color='blue', bottom=g_tesd[-1]+g_FC[-1])
bar1 = ax2.bar(hours, P_solar_heat, bar_width, label='Solar thermal energy', color='skyblue', bottom=g_tesd[-1]+g_FC[-1]+g_EL[-1])

bar4 = ax2.bar(hours, -g_tesc[-1], bar_width, label='$g^{tesc}_t$(<0)', color='salmon', bottom=0)
bar5 = ax2.bar(hours, -g_AC[-1], bar_width, label='$g^{AC}_t$(<0)', color='purple', bottom=-g_tesc[-1])

ax2.plot(hours, Q_HD, label='$Q^{HD}_t$', color='black')
ax2.set_xlabel('Hour')
ax2.set_ylabel('Power (kW)')
ax2.set_title('Hourly heat energy input and consumption')
ax2.set_xticks(hours)
ax2.set_xticklabels(hours)
ax2.legend()
plt.show()

In [ ]:
# 冷能来源与消耗策略
fig3, ax3 = plt.subplots(figsize=(10, 6), dpi=80)

bar1 = ax3.bar(hours, q_AC[-1], bar_width, label='$q^{AC}_t$', color='purple')
bar2 = ax3.bar(hours, q_cssd[-1], bar_width, label='$q^{cssd}_t$(>0)', color='blue')
bar3 = ax3.bar(hours, q_cssc[-1], bar_width, label='$q^{cssc}_t$(<0)', color='blue')
ax3.plot(hours, Q_CD, label='$Q^{CD}_t$', color='black')
ax3.set_xlabel('Hour')
ax3.set_ylabel('Power (kW)')
ax3.set_title('Hourly cooling energy input and consumption')
ax3.set_xticks(hours)
ax3.set_xticklabels(hours)
ax3.legend()
plt.show()